In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "collapsed_sections": [],
      "include_colab_link": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/gist/dellis23/9816c333a80ca0b5aa9a271e2d63a157/iree-torch-bert-test.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "# Introduction\n",
        "\n",
        "In this notebook, we will take a PyTorch Hugging Face BERT model and compile it down to a format executable by IREE.  We will then demonstrate the significantly reduced runtime size.  Additional features of IREE can be found on the [IREE homepage](https://iree-org.github.io/iree/#key-features)."
      ],
      "metadata": {
        "id": "ZvZ62ewxpi9L"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "# Package Installation\n",
        "\n",
        "To install `torch-mlir` (required to compile the model to a format processable by IREE), your Python version must be 3.9 or 3.10.\n",
        "\n",
        "As of September 2022, Colab only runs on 3.7.  You must use a [local Colab runtime](https://research.google.com/colaboratory/local-runtimes.html) with the correct Python version for this notebook to work correctly."
      ],
      "metadata": {
        "id": "df1pAY3poNEq"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "import platform\n",
        "assert platform.python_version().startswith('3.9.') or platform.python_version().startswith('3.10.')"
      ],
      "metadata": {
        "id": "yHFrHZ1DfrT0"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "%%capture\n",
        "!pip install -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html torch==1.13.0.dev20220913+cpu\n",
        "!pip install https://github.com/llvm/torch-mlir/releases/download/snapshot-20220913.595/torch_mlir-20220913.595-cp310-cp310-linux_x86_64.whl\n",
        "!pip install iree-compiler iree-runtime iree-tools-tf -f https://github.com/iree-org/iree/releases\n",
        "!pip install git+https://github.com/iree-org/iree-torch.git\n",
        "!pip install transformers"
      ],
      "metadata": {
        "id": "-v_vEvT8V1La"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "# Model Setup"
      ],
      "metadata": {
        "id": "a9iMOfulobtY"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "## TODO: Give a brief explanation of why this wrapping is necessary."
      ],
      "metadata": {
        "id": "iaTqNHjpoyWl"
      }
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "a9i_RnThVpxU"
      },
      "outputs": [],
      "source": [
        "import torch\n",
        "import torch_mlir\n",
        "import iree_torch\n",
        "\n",
        "from transformers import AutoTokenizer, AutoModelForSequenceClassification\n",
        "\n",
        "\n",
        "def prepare_sentence_tokens(hf_model: str, sentence: str):\n",
        "    tokenizer = AutoTokenizer.from_pretrained(hf_model)\n",
        "    return torch.tensor([tokenizer.encode(sentence)])\n",
        "\n",
        "\n",
        "class OnlyLogitsHuggingFaceModel(torch.nn.Module):\n",
        "    \"\"\"Wrapper that returns only the logits from a HuggingFace model.\"\"\"\n",
        "\n",
        "    def __init__(self, model_name: str):\n",
        "        super().__init__()\n",
        "        self.model = AutoModelForSequenceClassification.from_pretrained(\n",
        "            model_name,  # The pretrained model name.\n",
        "            # The number of output labels--2 for binary classification.\n",
        "            num_labels=2,\n",
        "            # Whether the model returns attentions weights.\n",
        "            output_attentions=False,\n",
        "            # Whether the model returns all hidden-states.\n",
        "            output_hidden_states=False,\n",
        "            torchscript=True,\n",
        "        )\n",
        "        self.model.eval()\n",
        "\n",
        "    def forward(self, input):\n",
        "        # Return only the logits.\n",
        "        return self.model(input)[0]\n",
        "\n",
        "\n",
        "# Suppress warnings\n",
        "import warnings\n",
        "warnings.simplefilter(\"ignore\")\n",
        "import os\n",
        "os.environ[\"TOKENIZERS_PARALLELISM\"] = \"true\""
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "# IREE Compilation\n",
        "\n",
        "Now, our PyTorch model can be compiled to MLIR and then to a format IREE is able to load and execute."
      ],
      "metadata": {
        "id": "ENzdaB8zogBj"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "# The HuggingFace model name to use\n",
        "model_name = \"philschmid/MiniLM-L6-H384-uncased-sst2\"\n",
        "\n",
        "# The sentence to run the model on\n",
        "sentence = \"The quick brown fox jumps over the lazy dog.\"\n",
        "\n",
        "print(\"Parsing sentence tokens.\")\n",
        "example_input = prepare_sentence_tokens(model_name, sentence)\n",
        "\n",
        "print(\"Instantiating model.\")\n",
        "model = OnlyLogitsHuggingFaceModel(model_name)\n",
        "\n",
        "print(\"Tracing model.\")\n",
        "traced = torch.jit.trace(model, example_input)\n",
        "\n",
        "print(\"Compiling with Torch-MLIR\")\n",
        "linalg_on_tensors_mlir = torch_mlir.compile(\n",
        "    traced,\n",
        "    example_input,\n",
        "    output_type=torch_mlir.OutputType.LINALG_ON_TENSORS)\n",
        "\n",
        "print(\"Compiling with IREE\")\n",
        "# Backend options:\n",
        "#\n",
        "# llvm-cpu - cpu, native code\n",
        "# vmvx - cpu, interpreted\n",
        "# vulkan - GPU for general GPU devices\n",
        "# cuda - GPU for NVIDIA devices\n",
        "iree_backend = \"llvm-cpu\"\n",
        "iree_vmfb = iree_torch.compile_to_vmfb(linalg_on_tensors_mlir, iree_backend)\n",
        "\n",
        "print(\"Loading in IREE\")\n",
        "invoker = iree_torch.load_vmfb(iree_vmfb, iree_backend)\n",
        "\n",
        "print(\"Running on IREE\")\n",
        "result = invoker.forward(example_input)\n",
        "print(\"RESULT:\", result)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "b6PqMZfXidbq",
        "outputId": "65404058-bd73-4401-b5cf-cfc1acbe9f67"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Parsing sentence tokens.\n",
            "Instantiating model.\n",
            "Tracing model.\n",
            "Compiling with Torch-MLIR\n",
            "Compiling with IREE\n",
            "Loading in IREE\n",
            "Running on IREE\n",
            "RESULT: tensor([[ 1.8574, -1.8036]])\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "We are now running our model on IREE.  The compiled version of this model can be saved, deployed, and executed independently of PyTorch.\n",
        "\n"
      ],
      "metadata": {
        "id": "x3njgRYXnacc"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "# Runtime Size Comparison\n",
        "\n",
        "One benefit of running a model on IREE is lightweight deployment.  The IREE runtime has a significantly smaller footprint than a full PyTorch install."
      ],
      "metadata": {
        "id": "zhrwdh9pmPIH"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "import os\n",
        "!du -sh {os.path.dirname(torch.__file__)}\n",
        "import iree.runtime as iree_runtime\n",
        "!du -sh {os.path.dirname(iree_runtime.__file__)}"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Ueyx5zvUly7M",
        "outputId": "61440bd9-d166-46d7-b1f6-ee922a61c355"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "713M\t/usr/local/google/home/danielellis/colab-test-venv/lib/python3.10/site-packages/torch\n",
            "4.0M\t/usr/local/google/home/danielellis/colab-test-venv/lib/python3.10/site-packages/iree/runtime\n"
          ]
        }
      ]
    }
  ]
}